\title{Satellite Imagery Search and Sort} \author{} \maketitle

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Intro-to-Landsat-imagery-catalog" data-toc-modified-id="Intro-to-Landsat-imagery-catalog-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro to Landsat imagery catalog</a></div><div class="lev2 toc-item"><a href="#Demo-codes-(won't-work)" data-toc-modified-id="Demo-codes-(won't-work)-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Demo codes (won't work)</a></div><div class="lev1 toc-item"><a href="#Convert-the-imagery-in-geotiff-into-numpy-arrays" data-toc-modified-id="Convert-the-imagery-in-geotiff-into-numpy-arrays-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Convert the imagery in geotiff into numpy arrays</a></div><div class="lev2 toc-item"><a href="#Multiband" data-toc-modified-id="Multiband-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Multiband</a></div><div class="lev2 toc-item"><a href="#Single-band" data-toc-modified-id="Single-band-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Single band</a></div><div class="lev1 toc-item"><a href="#Find-cloud-cover-over-satellite-imagery" data-toc-modified-id="Find-cloud-cover-over-satellite-imagery-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find cloud cover over satellite imagery</a></div><div class="lev2 toc-item"><a href="#detecting-lowest-cloud-cover-imagery" data-toc-modified-id="detecting-lowest-cloud-cover-imagery-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>detecting lowest cloud cover imagery</a></div>

# Intro to Landsat imagery catalog

1. catalog from [GCP](https://storage.googleapis.com/gcp-public-data-landsat/index.csv.gz)

## Demo codes (won't work)

In [ ]:
import pandas as pd
db=pd.read_csv('index.csv')
db.head()
db1=db[db.SPACECRAFT_ID=='LANDSAT_8']
db1.info()

db1['date'] = pd.to_datetime(db1['DATE_ACQUIRED'])  
mask = (db1['date'] > '2015-1-1') & (db1['date'] <= '2015-12-31')
db2 = db1.loc[mask]

db3=db2[(db2['WRS_PATH']==148)&(db2['WRS_ROW']==47)]
db3

In [ ]:
db1.sort_values('date')

In [ ]:
import os
import pandas as pd

for city in cities:
    pathrow3=pathrow2[pathrow2.city==city]
    newpath = '/home/sunbird/landsat/'+city 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for index, row in pathrow3.iterrows():
        os.popen("gsutil -m cp -r "+row['BASE_URL']+" /home/sunbird/landsat/"+city+"/").readlines()

In [ ]:
import fiona
import rasterio
import rasterio.tools.mask
import glob
import os
import ntpath


with fiona.open("Mumbai_extent_32643.shp", "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]


lcfolders='mumbai/LC81480472016284LGN00'
    
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

tifiles=glob.glob(lcfolders+'/*.TIF')
for tifile in tifiles:
    shpfiles2=(os.path.splitext(tifile)[0])
    shpfile1=path_leaf(shpfiles2)
    with rasterio.open(tifile) as src:
        out_image, out_transform = rasterio.tools.mask.mask(src, features,crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff","height": out_image.shape[1],"width": out_image.shape[2],"transform": out_transform})
        with rasterio.open('/home/sunbird/treecover/Msubset/'+shpfile1+".tif", "w", **out_meta) as dest:
            dest.write(out_image)

# Convert the imagery in geotiff into numpy arrays

In [ ]:
import rasterio

## Multiband

In [ ]:
dataset = rasterio.open("../data/raster/Mumbai_2016.tif")
data=dataset.read(1)

## Single band

In [ ]:
dataset = rasterio.open("../data/raster/Mumbai_2016.tif")
data=dataset.read(1)

In [ ]:
data.shape

# Find cloud cover over satellite imagery
1. Apply fmask algorthm
1. Algorithm based [on](http://pythonfmask.org/en/latest/)

In [ ]:
import os
import sys
import argparse

from fmask import landsatangles
from fmask import config

from rios import fileinfo
from rios import applier
import numpy
import glob
from pyproj import Proj, transform
import datetime



def sunAnglesForExtent(imgInfo, mtlInfo):
    """
    Return array of sun azimuth and zenith for each of the corners of the image
    extent. Note that this is the raster extent, not the corners of the swathe.

    The algorithm used here has been copied from the 6S possol() subroutine. The
    Fortran code I copied it from was .... up to the usual standard in 6S. So, the
    notation is not always clear.

    """
    inProj = Proj(init='epsg:32643')
    outProj = Proj(init='epsg:4326')
    cornerLatLong = imgInfo.getCorners()
    (ul_long, ul_lat, ur_long, ur_lat, lr_long, lr_lat, ll_long, ll_lat) = cornerLatLong
    ul_long1, ul_lat1=transform(inProj,outProj,ul_long, ul_lat)
    ur_long1, ur_lat1=transform(inProj,outProj,ur_long, ur_lat)
    lr_long1, lr_lat1=transform(inProj,outProj,lr_long, lr_lat,)
    ll_long1, ll_lat1=transform(inProj,outProj,ll_long, ll_lat)
    pts = numpy.array([
        [ul_long1, ul_lat1],
        [ur_long1, ur_lat1],
        [ll_long1, ll_lat1],
        [lr_long1, lr_lat1]
    ])
    longDeg = pts[:, 0]
    latDeg = pts[:, 1]
    # Date/time in UTC
    dateStr = mtlInfo['DATE_ACQUIRED']
    timeStr = mtlInfo['SCENE_CENTER_TIME'].replace('Z', '')
    ymd = [int(i) for i in dateStr.split('-')]
    dateObj = datetime.date(ymd[0], ymd[1], ymd[2])
    julianDay = (dateObj - datetime.date(ymd[0], 1, 1)).days + 1
    juldayYearEnd = (datetime.date(ymd[0], 12, 31) - datetime.date(ymd[0], 1, 1)).days + 1
    # Julian day as a proportion of the year
    jdp = julianDay / juldayYearEnd
    # Hour in UTC
    hms = [float(x) for x in timeStr.split(':')]
    hourGMT = hms[0] + hms[1] / 60.0 + hms[2] / 3600.0
    (sunAz, sunZen) = landsatangles.sunAnglesForPoints(latDeg, longDeg, hourGMT, jdp)
    sunAngles = numpy.vstack((sunAz, sunZen)).T
    return sunAngles


def getCtrLatLong1(imgInfo):
    """
    Return the lat/long of the centre of the image as
        (ctrLat, ctrLong)

    """
    cornerLatLong = imgInfo.getCorners()
    inProj = Proj(init='epsg:32643')
    outProj = Proj(init='epsg:4326')
    (ul_long, ul_lat, ur_long, ur_lat, lr_long, lr_lat, ll_long, ll_lat) = cornerLatLong
    ul_long1, ul_lat1=transform(inProj,outProj,ul_long, ul_lat)
    ur_long1, ur_lat1=transform(inProj,outProj,ur_long, ur_lat)
    lr_long1, lr_lat1=transform(inProj,outProj,lr_long, lr_lat)
    ll_long1, ll_lat1=transform(inProj,outProj,ll_long, ll_lat)
    ctrLat = numpy.array([ul_lat, ur_lat, lr_lat, ll_lat]).mean()
    ctrLong = numpy.array([ul_long, ur_long, lr_long, ll_long]).mean()
    return (ctrLat, ctrLong)



def makeAnglesImage(templateimg, outfile, nadirLine, extentSunAngles, satAzimuth, imgInfo):
    """
    Make a single output image file of the sun and satellite angles for every
    pixel in the template image.

    """
    imgInfo  = fileinfo.ImageInfo(templateimg)
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()
    otherargs = applier.OtherInputs()
    controls = applier.ApplierControls()
    infiles.img = templateimg
    outfiles.angles = outfile
    (ctrLat, ctrLong) = getCtrLatLong1(imgInfo)
    otherargs.R = landsatangles.localRadius(ctrLat)
    otherargs.nadirLine = nadirLine
    otherargs.xMin = imgInfo.xMin
    otherargs.xMax = imgInfo.xMax
    otherargs.yMin = imgInfo.yMin
    otherargs.yMax = imgInfo.yMax
    otherargs.extentSunAngles = extentSunAngles
    otherargs.satAltitude = 705000      # Landsat nominal altitude in metres
    otherargs.satAzimuth = satAzimuth
    otherargs.radianScale = 100        # Store pixel values as (radians * radianScale)
    controls.setStatsIgnore(500)
    applier.apply(landsatangles.makeAngles, infiles, outfiles, otherargs, controls=controls)
    

os.chdir('/home/sunbird/Documents/FOSS-Python-GeospatialAnalysis/data/raster/')
os.popen("gdal_merge.py -separate -o "+'ref_'+'mumimg'+".tif"+" LC*_B[1-7,9].tif").readlines()
os.popen("gdal_merge.py -separate -o "+'thermal_'+'mumimg'+".tif"+" LC*_B1[0,1].tif").readlines()
mtlInfo = config.readMTLFile('LC81480472016284LGN00_MTL.txt')
imgInfo = fileinfo.ImageInfo('ref_mumimg.tif')
corners = landsatangles.findImgCorners('ref_mumimg.tif', imgInfo)
nadirLine = landsatangles.findNadirLine(corners)
extentSunAngles = sunAnglesForExtent(imgInfo, mtlInfo)
satAzimuth = landsatangles.satAzLeftRight(nadirLine)
makeAnglesImage('ref_mumimg.tif', 'angles_mumimg.tif',nadirLine, extentSunAngles, satAzimuth, imgInfo)
#os.popen("fmask_usgsLandsatSaturationMask.py -i "+"ref_"+'mumimg'+".tif"+" -m *_MTL.txt -o "+"saturationmask_"+'mumimg'+".tif").readlines()
#os.popen("fmask_usgsLandsatTOA.py -i "+"ref_"+'mumimg'+".tif"+" -m *_MTL.txt -z angles_"+'mumimg'+".tif "+"-o toa_"+'mumimg'+".tif").readlines()
#os.popen("fmask_usgsLandsatStacked.py -t "+"thermal_"+'mumimg'+".tif"+" -a "+"toa_"+'mumimg'+".tif"+" -m *_MTL.txt -z angles_"+'mumimg'+".tif"+" -s "+"saturationmask_"+'mumimg'+".tif"+" -o "+"cloud_"+'mumimg'+".tif").readlines()



In [ ]:
os.popen("fmask_usgsLandsatSaturationMask.py -i ref_mumimg.tif -m *_MTL.txt -o saturationmask.tif").readlines()
os.popen("fmask_usgsLandsatTOA.py -i ref_mumimg.tif -m *_MTL.txt -z angles_mumimg.tif -o toa.tif").readlines()
os.popen("fmask_usgsLandsatStacked.py -t thermal_mumimg.tif -a toa.tif -m *_MTL.txt -z angles_mumimg.tif -s saturationmask.tif -o cloud.tif").readlines()

## detecting lowest cloud cover imagery

In [ ]:
import os
import glob
import rasterio
import numpy as np

os.chdir('/home/sunbird/Documents/FOSS-Python-GeospatialAnalysis/data/raster/')
dataset = rasterio.open("cloud.tif")
data=dataset.read(1)
unique, counts = np.unique(data, return_counts=True)
dattif=dict(zip(unique, counts))
dattif1={x: dattif[x] for x in [2,3] if x in dattif}
percloud=round(((float(sum([i for i in dattif1.values()])*30*30)/(1000*1000))) / ((sum([i for i in dattif.values()])*30*30)/(1000*1000)) *100)
print percloud